In [1]:
# add to path
import sys
if '../' not in sys.path:
    sys.path.insert(0, '../')
sys.path

['../',
 '/home/jimmyyao/repos/rev-stable-diffusion-2/notebooks',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python39.zip',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/lib-dynload',
 '',
 '/home/jimmyyao/.local/lib/python3.9/site-packages',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages',
 '/home/jimmyyao/repos/segment-anything',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/certifi-2022.12.7-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/wheel-0.40.0-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/setuptools-67.6.1-py3.9.egg',
 '/home/jimmyyao/repos/GenerativeImage2Text']

In [2]:
from samgit.inference import prepare_model, forward

PRETRAINED_MODEL = "../notebooks/output/SAMGIT/epoch1/model.pt"
SAM_MODEL_TYPE="vit_h"
SAM_CHECKPOINT="../models/sam_vit_h_4b8939.pth"
# TOKENIZER_PRETRAINED="/kaggle/input/bert-base-uncased"
model, mask_generator, tokenizer, param = prepare_model(PRETRAINED_MODEL,
                                                        sam_model_type=SAM_MODEL_TYPE,
                                                        # tokenizer_pretrained=TOKENIZER_PRETRAINED,
                                                        sam_checkpoint=SAM_CHECKPOINT
                                                       )

/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def caption_img(img_path: str) -> str:
    result = forward(model, mask_generator, tokenizer, param, img_path)
    return tokenizer.decode(result['predictions'][0].tolist(), skip_special_tokens=True)

def caption_img_batch(img_paths: list) -> list:
    result = forward(model, mask_generator, tokenizer, param, img_paths)
    return [tokenizer.decode(i.tolist(), skip_special_tokens=True) for i in result['predictions']]

In [4]:
# get data
import os
PART1_IMGS_PATH = "../datasets/sampleeval"

#csv
import pandas as pd
df = pd.read_csv("../datasets/sampleeval/prompts.csv")
df.head()

,imgId,prompt
0,20057f34d,hyper realistic photo of very friendly and dys...
1,227ef0887,"ramen carved out of fractal rose ebony, in the..."
2,92e911621,ultrasaurus holding a black bean taco in the w...
3,a4e1c55a9,a thundering retro robot crane inks on parchme...
4,c98f79f71,"portrait painting of a shimmering greek hero, ..."


In [5]:
img_paths = [os.path.join(PART1_IMGS_PATH, df.iloc[idx]['imgId']+".png") for idx in range(len(df))]
img_paths

['../datasets/sampleeval/20057f34d.png',
 '../datasets/sampleeval/227ef0887.png',
 '../datasets/sampleeval/92e911621.png',
 '../datasets/sampleeval/a4e1c55a9.png',
 '../datasets/sampleeval/c98f79f71.png',
 '../datasets/sampleeval/d8edf2e40.png',
 '../datasets/sampleeval/f27825b2c.png']

In [6]:
# from samgit.train import forward as trian_forward
# from generativeimage2text.common import Config

# cfg = {
#         'crop_region_extend_in_datatransform': 4,
#         'data_normalize': 'clip',
#         'train_crop_size': 224,
#         'input_small_scale': 0.8,
#         'no_color_jitter': True,
#         'no_flip': True,
#         'no_aspect_dist': True,
#         'interpolation': 'bicubic',
#         'min_size_range32': [160, 224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
#         # 'min_size_range32': [224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
#         'patch_size': 16,
#         'train_transform': 'vitp',
#     }
# cfg = Config(cfg, {})

# trian_forward(model, mask_generator, tokenizer, cfg, param, img_paths, df['prompt'].tolist())

In [7]:
def generate(result, tokenizer):
    captions = tokenizer.batch_decode(result['predictions'], skip_special_tokens=True)
    return captions

In [8]:
# result = forward(model, mask_generator, tokenizer, param, img_paths, main_crop_size=160)
# generate(result, tokenizer)

In [9]:
# new_predictions = []
# for img_path in img_paths:
#     r = forward(model, mask_generator, tokenizer, param, img_path, main_crop_size=160)
#     new_predictions.append(r['predictions'][0])
# generate({'predictions':new_predictions}, tokenizer)

In [10]:
# result = forward(model, mask_generator, tokenizer, param, img_paths, main_crop_size=224)
# generate(result, tokenizer)

In [11]:
import os

r = forward(model, mask_generator, tokenizer, param, os.path.join(PART1_IMGS_PATH, f"2.png"), main_crop_size=224)
generate(r, tokenizer)

['cat playing with a ball']

In [82]:
new_predictions = []
for img_path in img_paths:
    r = forward(model, mask_generator, tokenizer, param, img_path, main_crop_size=224)
    new_predictions.append(r['predictions'][0])

gen_captions = generate({'predictions':new_predictions}, tokenizer)

['a black hole in the middle of the world, digital art',
 'a wood carving of a man',
 'a dinosaur in the middle of a forest',
 'a drawing of a robot',
 'painting of a man with a dragonfly head',
 'an astronaut walking through a cherry blossom garden',
 'a man eating pizza at a mcdonalds']

In [83]:
df['gen_caption'] = gen_captions

In [84]:
df

,imgId,prompt,gen_caption
0,20057f34d,hyper realistic photo of very friendly and dys...,"a black hole in the middle of the world, digit..."
1,227ef0887,"ramen carved out of fractal rose ebony, in the...",a wood carving of a man
2,92e911621,ultrasaurus holding a black bean taco in the w...,a dinosaur in the middle of a forest
3,a4e1c55a9,a thundering retro robot crane inks on parchme...,a drawing of a robot
4,c98f79f71,"portrait painting of a shimmering greek hero, ...",painting of a man with a dragonfly head
5,d8edf2e40,an astronaut standing on a engaging white rose...,an astronaut walking through a cherry blossom ...
6,f27825b2c,Kaggle employee Phil at a donut shop ordering ...,a man eating pizza at a mcdonalds


In [85]:
df.to_csv("submission.csv", index=False)

In [86]:
from sentence_transformers import SentenceTransformer
st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [87]:
df['prompt'].to_list()

['hyper realistic photo of very friendly and dystopian crater',
 'ramen carved out of fractal rose ebony, in the style of hudson river school',
 'ultrasaurus holding a black bean taco in the woods, near an identical cheneosaurus',
 'a thundering retro robot crane inks on parchment with a droopy french bulldog',
 'portrait painting of a shimmering greek hero, next to a loud frill-necked lizard',
 'an astronaut standing on a engaging white rose, in the midst of by ivory cherry blossoms',
 'Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclaims "Donuts. It\'s what\'s for dinner!"']

In [88]:
# new_df = pd.DataFrame({'imgId': df['imgId'].to_list(), 'gen_caption': df['gen_caption'].to_list()})
# new_df.to_csv("submission.csv", index=False)

In [89]:
df['gen_caption'].to_list()

['a black hole in the middle of the world, digital art',
 'a wood carving of a man',
 'a dinosaur in the middle of a forest',
 'a drawing of a robot',
 'painting of a man with a dragonfly head',
 'an astronaut walking through a cherry blossom garden',
 'a man eating pizza at a mcdonalds']

In [90]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

prompt_embeddings = st_model.encode(df['prompt'].to_list())
gen_caption_embeddings = st_model.encode(df['gen_caption'].to_list())

print(cos(torch.tensor(prompt_embeddings), torch.tensor(gen_caption_embeddings)).mean().item())


0.4559473395347595


# SAM Crops Inference

In [129]:
from transformers import AutoProcessor, AutoModelForCausalLM

processor = AutoProcessor.from_pretrained("microsoft/git-base")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")

from PIL import Image

def generate_caption(image: Image):
    # image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs)
    return processor.decode(outputs[0], skip_special_tokens=True, max_length=40)

image = Image.open(img_paths[0]).convert("RGB")
generate_caption(image)

/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'a circle in the middle of the earth'

In [134]:
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

# unconditional image captioning
from PIL import Image

def generate_caption_blip(image: Image):
    # image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to("cuda")
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True, max_length=40)

image = Image.open(img_paths[0]).convert("RGB")
generate_caption_blip(image)

/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'arafed view of a circular hole in the middle of a desert'

In [214]:
def sam_image_crops(image, mask_generator, top_n_bbox, h_w_ratio=2.5):
    masks = mask_generator.generate(image)
    sorted_anns = sorted(masks, key=(lambda x: x['predicted_iou']), reverse=True)
    
    crops = []
    for ann in sorted_anns:
        x0, y0, x1, y1 = ann['bbox']
        if x0 > x1: 
            x0, x1 = x1, x0
        if y0 > y1: 
            y0, y1 = y1, y0
        
        height, width = y1-y0, x1-x0
        # print(crop.shape)
        ratio = height/(width+1e-6)
        if width > height:
            ratio = width/(height+1e-6)
        
        if height < 10 or width < 10:
            continue
        
        if ratio > h_w_ratio: # throw out images that are too thin, they don't provide much info
            continue

        crops.append(image[y0:y1, x0:x1, :])
        if len(crops) >= top_n_bbox:
            break
        
    return crops

from torch.cuda.amp import autocast
import cv2
def caption_img_sam(image: str, caption_main=True) -> str:
    with torch.no_grad():
        with autocast(dtype=torch.float16):
            image = cv2.imread(image)
            crops = sam_image_crops(image, mask_generator, 2, h_w_ratio=2.5) # top 2 iou with h_w_ratio
            
            crop_captions = []
            for crop in crops:
                if 0 in crop.shape:
                    continue
                crop = Image.fromarray(crop[:,:,::-1])
                crop_captions.append(generate_caption_blip(crop))
    # return ""
    # print(len(crop_captions))
    
        image_PIL = Image.fromarray(image[:,:,::-1])
        caption = generate_caption_blip(image_PIL)
    
    
    crop_captions = ", ".join(crop_captions)
    if crop_captions == "":
        return caption
    else:
        return f"{caption}, {crop_captions}"

In [150]:
from tqdm import tqdm

# get data
import os
IMGS_PATH = "../datasets/cococaps/val2014"

#json
import json

# Opening JSON file
with open('../datasets/cococaps/captions_val2014.json') as f:
    data = json.load(f)
    
from torch.utils.data import DataLoader, Dataset

class COCOEvalDataset(Dataset):
    def __init__(self, data, db_base_path):
        self.data = data
        self.captions = {a['image_id']:a for a in data['annotations']}
        
        self.db_base_path = db_base_path
        
    def __len__(self):
        return len(self.data['images'])
    
    def __getitem__(self, idx):
        image_file = os.path.join(self.db_base_path, self.data['images'][idx]['file_name'])
        image_id = self.data['images'][idx]['id']
        cap = self.captions[data['images'][idx]['id']]['caption']
        return image_id, image_file, cap
    

cocoeval_dataset = COCOEvalDataset(data, IMGS_PATH)
cocoeval_dataloader = DataLoader(cocoeval_dataset, batch_size=1, shuffle=False, num_workers=0)
    
pbar = tqdm(cocoeval_dataloader)

# fake_val2014 = {'image_id':-1, "caption":"fake caption"}

captions_val2014_fakecap_results = []
for image_id, image_file, cap in pbar:
    pbar.set_description(f"[step {pbar.n + 1}|{image_id[0].item()}] {cap[0]}\t")
    
    gen_caption = caption_img_sam(image_file[0])
    captions_val2014_fakecap_results.append({'image_id': image_id[0].item(), 'caption': gen_caption})
    if (pbar.n + 1) % 10000 == 0:
        with open(f'../datasets/cococaps/captions_sam+blip{pbar.n + 1}.json', 'w') as f:
            json.dump(captions_val2014_fakecap_results, f)

[step 1|391895] A man in a red shirt and a red hat is on a motorcycle on a hill side.	:   0%|          | 0/40504 [00:00<?, ?it/s]/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
[step 76|192440] A shelf with hygiene products in a bathroom.a pedestal sink.	:   0%|          | 75/40504 [01:07<10:31:48,  1.07it/s]                                        
[step 76|192440] A shelf with hygiene products in a bathroom.          
[step 185|521634] There is an image of an outdoor area. treet signs on a pole	:   0%|          | 184/40504 [02:45<10:06:45,  1.11it/s]                                      

[step 185|521634] There is an image of an outdoor area. ]               

ValueError: mean must have 1 elements if it is an iterable, got 3

In [225]:
pbar2 = tqdm(range(12652, len(cocoeval_dataset)), initial=12652)
for i in pbar2:

    image_id, image_file, cap = cocoeval_dataset[i]
    pbar.set_description(f"[step {pbar.n + 1}|{image_id}] {cap}\t")

    gen_caption = caption_img_sam(image_file)
    captions_val2014_fakecap_results.append({'image_id': image_id, 'caption': gen_caption})
    if (pbar.n + 1) % 10000 == 0:
        with open(f'../datasets/cococaps/captions_sam+blip{pbar.n + 1}.json', 'w') as f:
            json.dump(captions_val2014_fakecap_results, f)
    

29520it [4:01:23,  1.16it/s]                             


KeyboardInterrupt: 

: 

In [ ]:
with open('../datasets/cococaps/captions_sam+blip.json', 'w') as f:
    json.dump(captions_val2014_fakecap_results, f)

In [136]:
gen_captions = []
for img_path in img_paths:
    cap = caption_img_sam(img_path)
    gen_captions.append(cap)
gen_captions

2
1
2
2
0
2
2


['arafed view of a circular hole in the middle of a desert, arafed view of a circular hole with a small amount of dirt, arafed view of a circular hole in the middle of a desert',
 'a close up of a wooden plate with a swirl design on it, a close up of a wooden plate with a flower on it',
 'cartoon dinosaur with a piece of cheese in its mouth in a forest, cartoon of a blue rat with a piece of cheese in his hand, cartoon dinosaur with a piece of cheese in its mouth',
 'there is a drawing of a robot holding a hammer, there is a drawing of a robot with a bow and arrow, there is a drawing of a robot with a clock on it',
 'painting of a man with a lizard on his head and a lizard on his shoulder',
 'arafed astronaut walking down a path in a park with cherry trees, astronaut in white spacesuit with american flag on the back, astronaut in white spacesuit with american flag and space shuttle in background',
 "there is a man standing in front of a counter with a pizza, there is a man sitting at a 

In [139]:
df['gen_caption'] = gen_captions
df

,imgId,prompt,gen_caption,cosine_similarity
0,20057f34d,hyper realistic photo of very friendly and dys...,arafed view of a circular hole in the middle o...,0.203885
1,227ef0887,"ramen carved out of fractal rose ebony, in the...",a close up of a wooden plate with a swirl desi...,0.253239
2,92e911621,ultrasaurus holding a black bean taco in the w...,cartoon dinosaur with a piece of cheese in its...,0.315348
3,a4e1c55a9,a thundering retro robot crane inks on parchme...,there is a drawing of a robot holding a hammer...,0.402890
4,c98f79f71,"portrait painting of a shimmering greek hero, ...",painting of a man with a lizard on his head an...,0.571044
5,d8edf2e40,an astronaut standing on a engaging white rose...,arafed astronaut walking down a path in a park...,0.439187
6,f27825b2c,Kaggle employee Phil at a donut shop ordering ...,there is a man standing in front of a counter ...,0.010855


In [140]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

prompt_embeddings = st_model.encode(df['prompt'].to_list())
gen_caption_embeddings = st_model.encode(df['gen_caption'].to_list())

print(cos(torch.tensor(prompt_embeddings), torch.tensor(gen_caption_embeddings)).mean().item())
df["cosine_similarity"] = cos(torch.tensor(prompt_embeddings), torch.tensor(gen_caption_embeddings)).tolist()

0.4599408507347107


In [141]:
df

,imgId,prompt,gen_caption,cosine_similarity
0,20057f34d,hyper realistic photo of very friendly and dys...,arafed view of a circular hole in the middle o...,0.409247
1,227ef0887,"ramen carved out of fractal rose ebony, in the...",a close up of a wooden plate with a swirl desi...,0.234214
2,92e911621,ultrasaurus holding a black bean taco in the w...,cartoon dinosaur with a piece of cheese in its...,0.382151
3,a4e1c55a9,a thundering retro robot crane inks on parchme...,there is a drawing of a robot holding a hammer...,0.452026
4,c98f79f71,"portrait painting of a shimmering greek hero, ...",painting of a man with a lizard on his head an...,0.705105
5,d8edf2e40,an astronaut standing on a engaging white rose...,arafed astronaut walking down a path in a park...,0.501832
6,f27825b2c,Kaggle employee Phil at a donut shop ordering ...,there is a man standing in front of a counter ...,0.535012


In [142]:
df.to_csv("SAM+BLIP.csv", index=False)